In [ ]:
# @title Install necessary libraries
!pip install openai ipywidgets graphviz ipython --quiet

# @title Load OpenAI API Credentials
from getpass import getpass
import os
import openai

# Check if key is already set (e.g., in Colab secrets)
api_key_env = os.environ.get('OPENAI_API_KEY')

if api_key_env:
    print("OpenAI API Key found in environment variables.")
    OPENAI_KEY = api_key_env
else:
    print("OpenAI API Key not found in environment variables.")
    # Fallback to getpass if not found
    try:
        OPENAI_KEY = getpass('Enter your OpenAI API Key: ')
        os.environ['OPENAI_API_KEY'] = OPENAI_KEY
        openai.api_key = OPENAI_KEY
        # Test the key with a simple call (optional but recommended)
        # client = openai.OpenAI()
        # client.models.list()
        print("OpenAI API Key configured.")
    except Exception as e:
        print(f"Failed to configure OpenAI API Key: {e}")
        OPENAI_KEY = None # Ensure it's None if setup failed

# Configure the OpenAI client (using the newer openai > 1.0 library structure)
if OPENAI_KEY:
    try:
      client = openai.OpenAI(api_key=OPENAI_KEY)
      print("OpenAI client initialized successfully.")
    except Exception as e:
      print(f"Error initializing OpenAI client: {e}")
      client = None
else:
    print("Cannot initialize OpenAI client: API Key is missing.")

# @title Import other required libraries
import ipywidgets as widgets
from IPython.display import display, clear_output, Markdown
import graphviz
import uuid # For generating unique node IDs
import json # For potentially storing/loading graphs
import time # For simulating delays if needed
import traceback # For better error reporting
import copy # For deep copying node data if needed
import json
from google.colab import files # For file upload/download

In [ ]:
# @title Node Graph Data Structure and LLM Function

# Global dictionary to store our nodes and their connections
# Structure Change: 'outputs' values are now LISTS of node IDs
# nodes = {
#     'node_id_1': {
#         'name': 'User Friendly Name', 'type': 'Button', 'options': {},
#         'outputs': {'on_click': ['node_id_2', 'node_id_4']} # <<< LIST HERE
#     },
#     'node_id_2': {
#         'name': 'Ask LLM', 'type': 'LLM Program', 'options': {'prompt': '...'},
#         'outputs': {'result': ['node_id_3']} # <<< LIST HERE
#      },
#      'node_id_3': {
#         'name': 'Show Result', 'type': 'Show', 'options': {'text_template': '...'},
#         'outputs': {} # No outputs from Show node
#      },
#      'node_id_4': {
#          'name': 'Show Another', 'type': 'Show', 'options': {'text_template': 'Button also triggered this!'},
#          'outputs': {}
#      }
# }
nodes = {}
editing_node_id = None # Track which node ID is currently being edited

# --- LLM Interaction Function (No Change) ---
def call_llm(prompt_text, model="gpt-3.5-turbo"):
    """Calls the OpenAI API to get a completion."""
    if not client:
        return "Error: OpenAI client not initialized. Please check API Key."
    try:
        # print(f"\nDEBUG: Calling LLM with prompt: '{prompt_text[:100]}...'") # Debug print
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": "You are a helpful assistant executing user instructions."},
                {"role": "user", "content": prompt_text}
            ]
        )
        result = response.choices[0].message.content.strip()
        # print(f"DEBUG: LLM Response received: '{result[:100]}...'") # Debug print
        return result
    except Exception as e:
        error_message = f"Error calling OpenAI API: {e}"
        print(f"DEBUG: {error_message}") # Debug print
        return f"Error: Could not get response from LLM. Details: {e}"

print("Node Graph Data Structure (with list outputs) and LLM function defined.")

In [ ]:
# @title Graph Visualization Function (Handles Multiple Connections)

graph_output = widgets.Output()

def visualize_graph():
    """Renders the current node graph using Graphviz, handling multiple outputs."""
    dot = graphviz.Digraph(comment='Node Graph', format='png')
    dot.attr(rankdir='LR')

    with dot.subgraph(name='cluster_nodes') as c:
        c.attr(label='Nodes', style='filled', color='lightgrey')
        for node_id, node_data in nodes.items():
            node_label = f"ID: {node_id}\nType: {node_data['type']}\nName: {node_data['name']}"
            # Add options preview if applicable
            if node_data['type'] == 'LLM Program':
                node_label += f"\nPrompt: {node_data['options'].get('prompt', '')[:30]}..."
            elif node_data['type'] == 'Show':
                 node_label += f"\nText: {node_data['options'].get('text_template', '')[:30]}..."

            # Highlight node being edited
            node_color = 'lightblue' if node_id == editing_node_id else 'white'
            c.node(node_id, label=node_label, shape='box', style='filled', fillcolor=node_color)

    # Add edges after all nodes are defined
    for node_id, node_data in nodes.items():
        for port_name, target_node_ids in node_data.get('outputs', {}).items():
            # Ensure target_node_ids is a list (even if loaded from old format)
            if isinstance(target_node_ids, str):
                target_node_ids = [target_node_ids] # Convert old format on the fly

            if isinstance(target_node_ids, list):
                for target_node_id in target_node_ids: # Iterate through the list
                    if target_node_id in nodes:
                        dot.edge(node_id, target_node_id, label=port_name)
                    else:
                        print(f"Warning: Connection from {node_id} ({port_name}) points to non-existent node {target_node_id}")
            # else: Handle potential data corruption? print warning?

    with graph_output:
        clear_output(wait=True)
        if not nodes:
            print("Graph is empty. Add some nodes!")
        else:
            try:
                display(dot)
            except graphviz.backend.execute.ExecutableNotFound:
                 display(Markdown("**Error:** `graphviz` executable not found. Rendering failed."))
            except Exception as e:
                 display(Markdown(f"**Error rendering graph:** {e}"))

print("Graph Visualization function updated for multiple connections.")
# Initial visualization (will be empty)
visualize_graph()

In [ ]:
# @title Node Definition, Editing, Removal, Save/Load UI (Added Interactive Input)

import ipywidgets as widgets
from IPython.display import display, clear_output, Markdown
import uuid
import graphviz
import json
from google.colab import files

# --- Global State ---
if 'nodes' not in globals(): nodes = {}
editing_node_id = None

# --- Output Widgets ---
edit_feedback_output = widgets.Output()
add_node_output = widgets.Output()
add_connection_output = widgets.Output()
save_load_output = widgets.Output()
if 'graph_output' not in globals(): graph_output = widgets.Output()
if 'visualize_graph' not in globals():
    def visualize_graph(): print("Error: visualize_graph not defined")

# --- VALID NODE TYPES --- Added 'Interactive Input'
VALID_NODE_TYPES = ['Button', 'LLM Program', 'Show', 'Text Input', 'Interactive Input']
NODE_TYPE_TO_INDEX = {t: i for i, t in enumerate(VALID_NODE_TYPES)}
INDEX_TO_NODE_TYPE = {i: t for i, t in enumerate(VALID_NODE_TYPES)}

# --- Helper: Get Node Choices ---
def get_node_choices():
    choices = {f"{data['name']} ({node_id})": node_id for node_id, data in nodes.items()}
    return choices if choices else {"(No nodes available)": None}

# --- Helper Functions for UI Updates ---
def update_all_dropdowns():
    dropdowns_to_update = ['source_node_dropdown', 'target_node_dropdown', 'select_node_edit_dropdown']
    if not all(d in globals() for d in dropdowns_to_update): return
    node_choices = get_node_choices()
    for dd_name in dropdowns_to_update:
        dropdown = globals()[dd_name]
        current_value = dropdown.value
        dropdown.options = node_choices
        if current_value in node_choices.values(): dropdown.value = current_value
        elif node_choices: dropdown.value = list(node_choices.values())[0]
        else: dropdown.value = None
    update_source_ports(None)

def update_source_ports(change):
    if 'source_port_dropdown' not in globals() or 'source_node_dropdown' not in globals(): return
    selected_node_id = source_node_dropdown.value
    port_options = []
    if selected_node_id and selected_node_id in nodes:
        node_type = nodes[selected_node_id]['type']
        if node_type == 'Button': port_options = ['on_click']
        elif node_type == 'LLM Program': port_options = ['result']
        elif node_type == 'Text Input': port_options = ['text_out']
        elif node_type == 'Interactive Input': port_options = ['user_input'] # Added port
    source_port_dropdown.options = port_options
    source_port_dropdown.value = port_options[0] if port_options else None

# --- Widgets for Node Creation/Editing ---
node_type_dropdown = widgets.Dropdown(options=VALID_NODE_TYPES, value='Button', description='Node Type:', style={'description_width': 'initial'})
node_name_text = widgets.Text(value='My Node', placeholder='Enter node name', description='Node Name:', style={'description_width': 'initial'})

# Option Boxes
button_options_box = widgets.VBox([])
llm_options_box = widgets.VBox([widgets.Textarea(value='Summarize.', placeholder='LLM prompt...', description='LLM Prompt:', layout={'width':'95%'}, style={'description_width':'initial'})])
show_options_box = widgets.VBox([widgets.Textarea(value='Output: {input}', placeholder='Show text...', description='Show Text:', layout={'width':'95%'}, style={'description_width':'initial'})])
text_input_options_box = widgets.VBox([widgets.Textarea(value='', placeholder='Static text...', description='Text Content:', layout={'width':'95%', 'height':'100px'}, style={'description_width':'initial'})])
interactive_input_options_box = widgets.VBox([widgets.Text(value='Enter value:', placeholder='Label for input field', description='Prompt Label:', layout={'width':'95%'}, style={'description_width':'initial'})]) # Added options

# Accordion including the new options box
options_accordion = widgets.Accordion(
    children=[button_options_box, llm_options_box, show_options_box, text_input_options_box, interactive_input_options_box], # Added interactive box
    selected_index=0
)
for i, node_type in enumerate(VALID_NODE_TYPES): options_accordion.set_title(i, f'{node_type} Options')

add_update_node_button = widgets.Button(description='Add Node', button_style='success', icon='plus')
cancel_edit_button = widgets.Button(description='Cancel Edit', button_style='warning', icon='times', visible=False)

# --- Widgets for Node Selection & Management ---
select_node_edit_dropdown = widgets.Dropdown(description='Select Node:', style={'description_width': 'initial'})
load_node_button = widgets.Button(description='Load for Edit', button_style='info', icon='edit')
remove_node_button = widgets.Button(description='Remove Selected Node', button_style='danger', icon='trash')

# --- Widgets for Adding Connections ---
source_node_dropdown = widgets.Dropdown(description='Source Node:', style={'description_width': 'initial'})
source_port_dropdown = widgets.Dropdown(description='Output Port:', style={'description_width': 'initial'})
target_node_dropdown = widgets.Dropdown(description='Target Node:', style={'description_width': 'initial'})
add_connection_button = widgets.Button(description='Add Connection', button_style='primary', icon='link')

# --- Widgets for Save/Load ---
save_filename_text = widgets.Text(value='graph.json', description='Filename:', style={'description_width': 'initial'})
save_graph_button = widgets.Button(description='Save Graph', button_style='success', icon='save')
load_graph_button = widgets.FileUpload(accept='.json', multiple=False, description='Load Graph', button_style='info', icon='upload')
clear_graph_button = widgets.Button(description='Clear Graph', button_style='danger', icon='recycle')

# --- Event Handlers ---

def reset_node_form(clear_name=True):
    global editing_node_id; editing_node_id = None
    node_type_dropdown.value = 'Button'
    if clear_name: node_name_text.value = 'My Button'
    # Reset all option fields
    llm_ta = next((w for w in llm_options_box.children if isinstance(w, widgets.Textarea)), None);
    if llm_ta: llm_ta.value = 'Summarize.'
    show_ta = next((w for w in show_options_box.children if isinstance(w, widgets.Textarea)), None);
    if show_ta: show_ta.value = 'Output: {input}'
    text_input_ta = next((w for w in text_input_options_box.children if isinstance(w, widgets.Textarea)), None);
    if text_input_ta: text_input_ta.value = ''
    interactive_input_text = next((w for w in interactive_input_options_box.children if isinstance(w, widgets.Text)), None); # Added reset
    if interactive_input_text: interactive_input_text.value = 'Enter value:'

    options_accordion.selected_index = NODE_TYPE_TO_INDEX['Button']
    add_update_node_button.description = 'Add Node'; add_update_node_button.button_style = 'success'; add_update_node_button.icon = 'plus'
    cancel_edit_button.visible = False; node_type_dropdown.disabled = False
    visualize_graph()

def on_cancel_edit_clicked(b):
    with add_node_output: clear_output(wait=True); print("Edit cancelled.")
    reset_node_form()

def handle_node_type_change(change):
    if editing_node_id is None:
        node_type = change.new; options_accordion.selected_index = NODE_TYPE_TO_INDEX.get(node_type, 0)
        if node_type == 'LLM Program': node_name_text.value = 'Ask LLM'
        elif node_type == 'Show': node_name_text.value = 'Display Result'
        elif node_type == 'Button': node_name_text.value = 'Start Button'
        elif node_type == 'Text Input': node_name_text.value = 'My Text'
        elif node_type == 'Interactive Input': node_name_text.value = 'Get User Input' # Added suggestion


node_type_dropdown.observe(handle_node_type_change, names='value')

def on_add_update_node_clicked(b):
    global editing_node_id
    node_type = node_type_dropdown.value
    node_name = node_name_text.value.strip() or f"Untitled_{node_type}"
    if not node_name:
        with add_node_output: clear_output(wait=True); print("❌ Node name empty.")
        return

    options = {}
    # Extract options based on type
    if node_type == 'LLM Program': options['prompt'] = next((w.value for w in llm_options_box.children if isinstance(w, widgets.Textarea)), '')
    elif node_type == 'Show': options['text_template'] = next((w.value for w in show_options_box.children if isinstance(w, widgets.Textarea)), '')
    elif node_type == 'Text Input': options['content'] = next((w.value for w in text_input_options_box.children if isinstance(w, widgets.Textarea)), '')
    elif node_type == 'Interactive Input': options['prompt_label'] = next((w.value for w in interactive_input_options_box.children if isinstance(w, widgets.Text)), '') # Added extraction

    with add_node_output: clear_output(wait=True)
    if editing_node_id:
        if editing_node_id not in nodes: print(f"❌ Error: Node {editing_node_id} not found."); reset_node_form(); return
        print(f"Updating '{nodes[editing_node_id]['name']}' ({editing_node_id})...")
        if nodes[editing_node_id]['type'] != node_type: print(f"⚠️ Type change ignored."); node_type = nodes[editing_node_id]['type']
        nodes[editing_node_id]['name'] = node_name; nodes[editing_node_id]['options'] = options
        print(f"✅ Node '{node_name}' updated.")
        reset_node_form(clear_name=True)
    else:
        new_node_id = str(uuid.uuid4())[:8]
        while new_node_id in nodes: new_node_id = str(uuid.uuid4())[:8]
        nodes[new_node_id] = {'name': node_name, 'type': node_type, 'options': options, 'outputs': {}}
        print(f"✅ Node '{node_name}' ({new_node_id}) added.")
        reset_node_form(clear_name=True)
    update_all_dropdowns(); visualize_graph()

def on_load_node_clicked(b):
    global editing_node_id; selected_id = select_node_edit_dropdown.value
    with edit_feedback_output: clear_output(wait=True)
    if not selected_id or selected_id not in nodes: print("❌ Select valid node."); reset_node_form(); return
    with edit_feedback_output: print(f"Loading '{nodes[selected_id]['name']}' ({selected_id})...")
    editing_node_id = selected_id; node_data = nodes[selected_id]
    node_name_text.value = node_data['name']; node_type_dropdown.value = node_data['type']
    node_type_dropdown.disabled = True; options_accordion.selected_index = NODE_TYPE_TO_INDEX.get(node_data['type'], 0)
    # Load options based on type
    if node_data['type'] == 'LLM Program': next((w for w in llm_options_box.children if isinstance(w, widgets.Textarea)), None).value = node_data['options'].get('prompt', '')
    elif node_data['type'] == 'Show': next((w for w in show_options_box.children if isinstance(w, widgets.Textarea)), None).value = node_data['options'].get('text_template', '')
    elif node_data['type'] == 'Text Input': next((w for w in text_input_options_box.children if isinstance(w, widgets.Textarea)), None).value = node_data['options'].get('content', '')
    elif node_data['type'] == 'Interactive Input': next((w for w in interactive_input_options_box.children if isinstance(w, widgets.Text)), None).value = node_data['options'].get('prompt_label', '') # Added loading

    add_update_node_button.description = 'Update Node'; add_update_node_button.button_style = 'info'; add_update_node_button.icon = 'save'
    cancel_edit_button.visible = True; visualize_graph()

def on_remove_node_clicked(b):
    # (Code unchanged - handles removal generically)
    global editing_node_id; node_to_remove_id = select_node_edit_dropdown.value
    with edit_feedback_output: clear_output(wait=True)
    if not node_to_remove_id or node_to_remove_id not in nodes: print("❌ Select valid node."); return
    removed_name = nodes[node_to_remove_id]['name']; print(f"Removing '{removed_name}' ({node_to_remove_id})...")
    del nodes[node_to_remove_id]
    nodes_to_check = list(nodes.keys())
    for check_id in nodes_to_check:
         if check_id in nodes:
            outputs_to_check = list(nodes[check_id]['outputs'].items())
            for port_name, target_ids in outputs_to_check:
                if isinstance(target_ids, str): target_ids = [target_ids]
                if isinstance(target_ids, list) and node_to_remove_id in target_ids:
                    print(f"  Removing conn from {check_id} [{port_name}] -> {node_to_remove_id}")
                    nodes[check_id]['outputs'][port_name].remove(node_to_remove_id)
                    if not nodes[check_id]['outputs'][port_name]: del nodes[check_id]['outputs'][port_name]
    print(f"✅ Node '{removed_name}' removed.")
    if editing_node_id == node_to_remove_id: print("  Cancelling edit."); reset_node_form()
    else: visualize_graph()
    update_all_dropdowns()

def on_add_connection_clicked(b):
    source_id = source_node_dropdown.value; port_name = source_port_dropdown.value; target_id = target_node_dropdown.value
    with add_connection_output: clear_output(wait=True)
    if not all([source_id, port_name, target_id]): print("❌ Select source, port, target."); return
    if source_id == target_id: print("❌ Cannot connect to self."); return
    if source_id not in nodes or target_id not in nodes: print("❌ Node missing."); return
    valid_ports = []; src_type = nodes[source_id]['type']
    # Define valid output ports for ALL types
    if src_type == 'Button': valid_ports = ['on_click']
    elif src_type == 'LLM Program': valid_ports = ['result']
    elif src_type == 'Text Input': valid_ports = ['text_out']
    elif src_type == 'Interactive Input': valid_ports = ['user_input'] # Added port check

    if port_name not in valid_ports: print(f"❌ Invalid port '{port_name}' for type {src_type}."); return
    output_dict = nodes[source_id]['outputs']
    if port_name not in output_dict: output_dict[port_name] = []
    if not isinstance(output_dict[port_name], list): output_dict[port_name] = [output_dict[port_name]]
    if target_id in output_dict[port_name]: print(f"ℹ️ Connection exists.")
    else: output_dict[port_name].append(target_id); print(f"✅ Connection: {nodes[source_id]['name']} [{port_name}] -> {nodes[target_id]['name']}")
    visualize_graph()

# --- Save/Load/Clear Handlers (Unchanged) ---
def on_save_graph_clicked(b):
    filename = save_filename_text.value;
    if not filename: filename = 'graph.json'
    if not filename.endswith('.json'): filename += '.json'
    try:
        json_string = json.dumps(nodes, indent=2)
        with open(filename, 'w') as f: f.write(json_string)
        files.download(filename)
        with save_load_output: clear_output(wait=True); print(f"✅ Saved & download started as '{filename}'.")
    except Exception as e:
        with save_load_output: clear_output(wait=True); print(f"❌ Error saving: {e}")

def on_load_graph_change(change):
    global nodes
    if not change['new']: return
    uploaded_filename = list(change['new'].keys())[0]
    uploaded_content = change['new'][uploaded_filename]['content']
    with save_load_output: clear_output(wait=True); print(f"Loading '{uploaded_filename}'...")
    try:
        json_string = uploaded_content.decode('utf-8')
        loaded_data = json.loads(json_string)
        if not isinstance(loaded_data, dict): raise ValueError("Not a valid JSON dictionary.")
        nodes = loaded_data
        print(f"✅ Graph loaded from '{uploaded_filename}'.")
        if editing_node_id: reset_node_form()
        update_all_dropdowns(); visualize_graph()
    except Exception as e:
        print(f"❌ Error loading graph: {e}"); nodes = {};
        update_all_dropdowns(); visualize_graph()
    finally:
        load_graph_button.value.clear()
        load_graph_button._counter = 0

def on_clear_graph_clicked(b):
    global nodes, editing_node_id
    with save_load_output:
        clear_output(wait=True)
        print("--> ACTION REQUIRED IN BROWSER <--")
        print("A confirmation prompt should appear.")
        from google.colab import output
        confirmed = output.eval_js('confirm("Clear the entire graph? Cannot be undone.")')
        if confirmed:
            clear_output(wait=True)
            nodes = {}; editing_node_id = None
            print("✅ Graph cleared.")
            reset_node_form(); update_all_dropdowns(); visualize_graph()
        else:
            clear_output(wait=True); print("Clear operation cancelled.")

# --- Attach Handlers ---
add_update_node_button.on_click(on_add_update_node_clicked)
cancel_edit_button.on_click(on_cancel_edit_clicked)
load_node_button.on_click(on_load_node_clicked)
remove_node_button.on_click(on_remove_node_clicked)
source_node_dropdown.observe(update_source_ports, names='value')
add_connection_button.on_click(on_add_connection_clicked)
save_graph_button.on_click(on_save_graph_clicked)
load_graph_button.observe(on_load_graph_change, names='value')
clear_graph_button.on_click(on_clear_graph_clicked)

# --- Layout the UI ---
manage_nodes_section = widgets.VBox([widgets.HTML("<b>Manage Nodes:</b>"), widgets.HBox([select_node_edit_dropdown, load_node_button, remove_node_button]), edit_feedback_output])
add_update_section = widgets.VBox([widgets.HTML("<hr><b>Add/Update Node Details:</b>"), node_type_dropdown, node_name_text, widgets.HTML("<i>Node Specific Options:</i>"), options_accordion, widgets.HBox([add_update_node_button, cancel_edit_button]), add_node_output])
connections_section = widgets.VBox([widgets.HTML("<hr><b>Add Connection:</b>"), source_node_dropdown, source_port_dropdown, target_node_dropdown, add_connection_button, add_connection_output])
save_load_section = widgets.VBox([widgets.HTML("<hr><b>Save/Load/Clear Graph:</b>"), widgets.HBox([save_filename_text, save_graph_button]), widgets.HBox([load_graph_button, clear_graph_button]), save_load_output])
graph_display_section = widgets.VBox([widgets.HTML("<hr><b>Current Graph:</b>"), graph_output])

# --- Initial Population ---
update_all_dropdowns()

# --- Display Combined UI ---
display(widgets.VBox([manage_nodes_section, add_update_section, connections_section, save_load_section, graph_display_section]))

print("Node Definition UI updated with 'Interactive Input' type.")

In [ ]:
# @title Play Mode Execution Logic and UI (FIX for Combining Interactive Inputs)

import ipywidgets as widgets
from IPython.display import display, clear_output, Markdown
import uuid
import traceback
import time
import collections # Needed for deque

# --- Global State Access ---
if 'nodes' not in globals(): nodes = {}
if 'client' not in globals(): client = None
if 'call_llm' not in globals():
    print("Warning: 'call_llm' function not found globally. Defining a placeholder.")
    def call_llm(prompt, model="gpt-3.5-turbo"):
        print(f"Placeholder LLM Call with prompt: {prompt[:100]}...")
        time.sleep(1); return "Placeholder LLM response."

# --- Output Widgets ---
play_mode_button = widgets.Button(description="🚀 Enter Play Mode", button_style='warning')
play_output = widgets.Output()
full_log_output = widgets.Output()

# --- Execution State ---
execution_queue = collections.deque()
run_states = {} # Stores state per run_id, e.g., {'run_id_1': {'interactive_inputs': {node_id: value}, 'executed_nodes': set()}}
active_run_id = None # Tracks the ID of the run being primarily processed (can be None)


# --- Helper to Add to Queue ---
def add_to_execution_queue(items):
    """Adds items (tuple or list of tuples) to the global queue."""
    global execution_queue
    if isinstance(items, list):
        execution_queue.extend(items)
    else:
        execution_queue.append(items)
    # print(f"DEBUG (add_to_execution_queue): Queue size now {len(execution_queue)}, Added: {items}")

# --- CORE QUEUE PROCESSING FUNCTION ---
def process_execution_queue():
    """Processes items from the global execution queue for the active run."""
    global active_run_id, execution_queue, run_states
    # print(f"DEBUG (process_execution_queue START): Active run: {active_run_id}, Queue size: {len(execution_queue)}")

    max_steps_per_call = len(nodes) + 10
    steps_processed = 0

    while execution_queue and steps_processed < max_steps_per_call:
        # Peek to check run_id
        current_node_id_peek, _, run_id_peek, _ = execution_queue[0]

        # Prioritize the active run if one is set
        if active_run_id and run_id_peek != active_run_id:
            # print(f"DEBUG: Next item run {run_id_peek} != active {active_run_id}, skipping step.")
            # Don't process items from other runs if one is active and waiting (e.g., for interactive)
            # This helps prevent interleaving runs in a confusing way
             break # Stop processing until the active run can continue

        # If no active run, process the next item regardless of its run_id
        # (This handles resuming potentially multiple waiting runs)
        current_node_id, current_input, run_id, _ = execution_queue.popleft() # We get executed_set from run_states

        # Ensure state exists for this run_id
        if run_id not in run_states:
            print(f"ERROR: State for run {run_id} not found! Skipping node {current_node_id}.")
            continue

        executed_in_run = run_states[run_id]['executed_nodes'] # Get the execution set for this run

        steps_processed += 1
        # print(f"DEBUG (DEQUEUE): Node {current_node_id}, Run {run_id}. Remaining: {len(execution_queue)}")

        # --- Standard checks ---
        if current_node_id in executed_in_run:
             with full_log_output: print(f"[{run_id}] Skipping node {current_node_id} - already processed.")
             continue
        executed_in_run.add(current_node_id) # Add to the run's specific set

        if current_node_id not in nodes:
            with play_output: display(Markdown(f"⚠️ **Error:** Node `{current_node_id}` not found."))
            with full_log_output: print(f"[{run_id}] Error: Node {current_node_id} not found.")
            continue

        node_data = nodes[current_node_id]
        node_name = node_data['name']
        node_type = node_data['type']

        with full_log_output: print(f"[{run_id}] Executing Node: {node_name} ({current_node_id}), Type: {node_type}")

        output_data = None
        port_to_follow = None
        next_nodes_info = [] # Stores tuples: (next_id, output_data, run_id) - no need for set here

        try:
            # --- Node Type Specific Logic ---
            if node_type == 'Button':
                output_data = current_input
                port_to_follow = 'on_click'

            elif node_type == 'Text Input':
                node_content = node_data['options'].get('content', '')
                # Static text nodes still output their content directly
                output_data = {'node_name': node_name, 'text': node_content}
                port_to_follow = 'text_out'
                with full_log_output: print(f"[{run_id}] Text Input node produced output.")

            elif node_type == 'Interactive Input':
                prompt_label = node_data['options'].get('prompt_label', f'{node_name}:')
                downstream_node_ids = []
                connected_ids = node_data.get('outputs', {}).get('user_input', [])
                if isinstance(connected_ids, str): connected_ids = [connected_ids]
                if isinstance(connected_ids, list): downstream_node_ids = connected_ids

                text_input_widget = widgets.Text(description=prompt_label, placeholder='Type & press Enter...', layout={'width': '80%'})

                # --- Define the submit handler ---
                # Pass the interactive node's ID to store the result correctly
                def handle_interactive_submit(widget, interactive_node_id=current_node_id, submit_run_id=run_id):
                    global active_run_id, run_states
                    user_text = widget.value; widget.disabled = True
                    with play_output: display(Markdown(f"✅ Input for '{nodes[interactive_node_id]['name']}': `{user_text}`"))
                    with full_log_output: print(f"[{submit_run_id}] User input for '{nodes[interactive_node_id]['name']}': '{user_text}'")

                    # Store the result in the run state
                    if submit_run_id in run_states:
                         run_states[submit_run_id]['interactive_inputs'][interactive_node_id] = user_text
                         print(f"DEBUG: Stored input for {interactive_node_id} in run {submit_run_id}")
                    else:
                         print(f"ERROR: Run state for {submit_run_id} not found when storing interactive input!")

                    items_to_add = []
                    # Queue the NEXT nodes, passing None as data (LLM will fetch)
                    for next_id in downstream_node_ids:
                        if next_id in nodes:
                             # Use the correct run_id captured by the handler closure
                             new_item = (next_id, None, submit_run_id, None) # Pass None for executed_set, will get from run_states
                             items_to_add.append(new_item)
                        else:
                             with full_log_output: print(f"[{submit_run_id}] Warning: Next node {next_id} not found.")

                    if items_to_add:
                        add_to_execution_queue(items_to_add)
                        # IMPORTANT: Set the active run ID to THIS run before processing queue,
                        # because processing might have stopped if multiple interactives were waiting.
                        active_run_id = submit_run_id
                        # print(f"DEBUG: Set active run to {active_run_id} before processing queue.")
                        process_execution_queue() # Resume processing for this run
                    # else: print(f"DEBUG: Interactive node {interactive_node_id} had no downstream.")
                # --- End of handler definition ---

                text_input_widget.on_submit(handle_interactive_submit) # Attach handler
                with play_output: display(Markdown(f"▶️ **Action Required for '{node_name}'**")); display(text_input_widget)
                with full_log_output: print(f"[{run_id}] Paused execution at '{node_name}', waiting...")

                # Make THIS run the active one while waiting
                active_run_id = run_id
                print(f"DEBUG: Paused. Set active run to {active_run_id}")
                break # Exit the 'while' loop, wait for user

            elif node_type == 'LLM Program':
                llm_node_prompt_template = node_data['options'].get('prompt', '')
                final_prompt_parts = []; gathered_inputs_formatted = []
                with full_log_output: print(f"[{run_id}] LLM Node '{node_name}' gathering inputs...")

                # 1. Scan for incoming INTERACTIVE inputs from run_states
                interactive_inputs_found = run_states.get(run_id, {}).get('interactive_inputs', {})
                with full_log_output: print(f"  > Checking stored interactive inputs for run {run_id}: {list(interactive_inputs_found.keys())}")
                for source_id, source_data in nodes.items(): # Check all nodes
                     # Find nodes connected TO this LLM node
                    source_outputs = source_data.get('outputs', {})
                    for port_name, target_ids in source_outputs.items():
                        if isinstance(target_ids, str): target_ids = [target_ids]
                        if current_node_id in target_ids: # Is current LLM node a target?
                            # Check if the source was an Interactive Input AND its result is stored
                            if source_data['type'] == 'Interactive Input' and source_id in interactive_inputs_found:
                                stored_text = interactive_inputs_found[source_id]
                                input_node_name = source_data['name']
                                gathered_inputs_formatted.append(f"Input from interactive node '{input_node_name}':\n{stored_text}")
                                with full_log_output: print(f"  > Found stored interactive input from '{input_node_name}' ({source_id})")

                # 2. Scan for connected STATIC Text Input nodes
                # with full_log_output: print(f"[{run_id}] LLM Node '{node_name}' scanning for connected STATIC Text Inputs...")
                for source_id, source_data in nodes.items():
                    if source_data['type'] == 'Text Input':
                        source_outputs = source_data.get('outputs', {})
                        text_out_targets = source_outputs.get('text_out', [])
                        if isinstance(text_out_targets, str): text_out_targets = [text_out_targets]
                        if current_node_id in text_out_targets:
                             text_content = source_data['options'].get('content', '')
                             text_node_name = source_data['name']
                             gathered_inputs_formatted.append(f"Input from static node '{text_node_name}':\n{text_content}")
                             with full_log_output: print(f"  > Found connected static text from '{text_node_name}' ({source_id})")


                # 3. Add direct input if it's not from Interactive/Static path handled above
                # (This logic might be redundant now, depends on desired behavior)
                # if isinstance(current_input, str) and current_input:
                #      is_handled = False # Check if this input was already gathered
                #      # ... logic to check if current_input matches a gathered one ...
                #      if not is_handled:
                #          gathered_inputs_formatted.append(f"Input from Previous Node:\n{current_input}")
                #          with full_log_output: print(f"  > Including direct string input.")

                # 4. Construct the context block
                if gathered_inputs_formatted:
                    context_block = "\n\n---\n".join(gathered_inputs_formatted)
                    final_prompt_parts.append("--- Input Context ---"); final_prompt_parts.append(context_block); final_prompt_parts.append("---------------------\n")
                else:
                     with full_log_output: print(f"[{run_id}] LLM node received no specific input context.")

                # 5. Add the LLM node's own instruction
                instruction = llm_node_prompt_template.replace('{input}', '').strip()
                if instruction: final_prompt_parts.append(f"User instruction: {instruction}")
                else: final_prompt_parts.append("User instruction: (None)")
                final_prompt = "\n".join(final_prompt_parts)

                # 6. Call LLM
                if client:
                    with full_log_output: print(f"[{run_id}] Calling LLM with combined prompt:\n{final_prompt[:600]}...")
                    output_data = call_llm(final_prompt)
                else:
                    output_data = "Error: OpenAI client not initialized."
                    with play_output: display(Markdown(f"⚠️ LLM Error: Client missing."))
                port_to_follow = 'result'

            elif node_type == 'Show':
                text_template = node_data['options'].get('text_template', 'Output: {input}')
                display_input_str = str(current_input) if current_input is not None else ""
                display_text = text_template.replace('{input}', display_input_str)
                with play_output: display(Markdown(f"**{node_name}:**\n\n```\n{display_text}\n```"))
                output_data = display_text
                port_to_follow = None

            else: # Unknown node type
                with play_output: display(Markdown(f"❓ Unknown Type: `{node_type}`"))
                with full_log_output: print(f"[{run_id}] Error: Unknown type {node_type}")
                port_to_follow = None

            # --- Find Next Nodes to Queue ---
            if port_to_follow and port_to_follow in node_data.get('outputs', {}):
                potential_next_ids = node_data['outputs'][port_to_follow]
                if isinstance(potential_next_ids, str): potential_next_ids = [potential_next_ids]
                if isinstance(potential_next_ids, list):
                    for next_id in potential_next_ids:
                         if next_id and next_id in nodes:
                            # Pass output_data from current node, and run_id. Set executed_set to None.
                            next_nodes_info.append((next_id, output_data, run_id, None))
                         elif next_id:
                             with full_log_output: print(f"[{run_id}] Warning: Target node {next_id} not found.")

        except Exception as e: # Catch errors during node execution
             tb_str = traceback.format_exc()
             error_md = Markdown(f"💥 Exec Error `{node_name}` ({current_node_id}):\n```\n{e}\n{tb_str[:500]}...\n```")
             with play_output: display(error_md)
             with full_log_output: print(f"[{run_id}] FATAL ERROR node {current_node_id}: {e}\n{tb_str}")
             continue

        # --- Queue Next Nodes (if any found) ---
        if next_nodes_info:
            add_to_execution_queue(next_nodes_info)

    # --- End of while execution_queue loop for this call ---
    # Check if the queue is empty AND we had an active run ID before clearing it
    if not execution_queue and active_run_id and active_run_id in run_states:
         # print(f"DEBUG: Run {active_run_id} finished and queue empty. Removing state.")
         del run_states[active_run_id] # Clean up state for completed run
         active_run_id = None # Clear active run ID only when queue is truly empty for it
    # elif not execution_queue:
    #     active_run_id = None # Clear if queue is empty even if no run was active
    # else: print(f"DEBUG: process_execution_queue loop exited. Queue size: {len(execution_queue)}")


# --- Function to Start a Flow ---
def execute_node_flow_start(start_node_id, input_data=None):
    """Sets up and starts processing a new flow execution."""
    global execution_queue, active_run_id, run_states
    new_run_id = str(uuid.uuid4())[:6]

    # Clean up old finished states (simple approach)
    # A more robust approach might use timestamps or explicit cleanup nodes
    runs_to_clear = [r_id for r_id in run_states if not any(item[2] == r_id for item in execution_queue)]
    for r_id in runs_to_clear:
        if r_id != active_run_id: # Don't clear the currently active run state yet
             print(f"DEBUG: Cleaning up state for old run {r_id}")
             del run_states[r_id]


    if active_run_id and execution_queue:
        print(f"Warning: Starting new run {new_run_id} while run {active_run_id} might be waiting.")

    active_run_id = new_run_id # Set the new active run
    # Initialize state for the new run
    run_states[active_run_id] = {
        'interactive_inputs': {},
        'executed_nodes': set()
    }


    with full_log_output: print(f"\n--- Triggering New Execution Run {active_run_id} from Node {start_node_id} ---")

    # Add starting item - pass None for executed_set initially, will be retrieved from run_states
    add_to_execution_queue([(start_node_id, input_data, active_run_id, None)])
    # print(f"DEBUG (execute_node_flow_start): Added start node {start_node_id} to queue for run {active_run_id}.")

    process_execution_queue() # Start processing


# --- Play Mode Button Handler ---
def on_play_mode_clicked(b):
    with play_output: clear_output(wait=True); display(Markdown("--- **Play Mode Activated** ---")); print("Scanning for Button nodes...")
    with full_log_output: clear_output(wait=True); print("--- Play Mode Log ---")
    buttons_to_display = []
    found_buttons = False
    for node_id, node_data in nodes.items():
        if node_data.get('type') == 'Button':
            found_buttons = True
            try:
                button_widget = widgets.Button(description=f"▶️ {node_data.get('name', 'Button')}", button_style='primary', tooltip=f"Trigger {node_id}")
                def handle_play_button_click(b_instance, n_id=node_id):
                    with play_output: display(Markdown(f"▶️ *Clicked '{nodes[n_id]['name']}'...*"))
                    try: execute_node_flow_start(n_id, input_data=None)
                    except Exception as e_flow_start:
                         tb_flow_start = traceback.format_exc()
                         err_start_md = Markdown(f"💥 Flow Start Error `{nodes[n_id]['name']}` ({n_id}):\n```\n{e_flow_start}\n{tb_flow_start[:500]}...\n```")
                         with play_output: display(err_start_md)
                         with full_log_output: print(f"FATAL ERROR starting flow from {n_id}: {e_flow_start}\n{tb_flow_start}")
                button_widget.on_click(handle_play_button_click)
                buttons_to_display.append(button_widget)
            except Exception as e_create:
                 print(f"ERROR creating button for {node_id}: {e_create}")
                 with play_output: display(Markdown(f"⚠️ Error creating button for `{node_data.get('name', node_id)}`"))
    with play_output:
        if not found_buttons: display(Markdown("No 'Button' nodes found."))
        else:
            # print(f"Displaying {len(buttons_to_display)} buttons...")
            for btn in buttons_to_display: display(btn)

play_mode_button.on_click(on_play_mode_clicked)

# Display UI Elements
display(widgets.VBox([
    play_mode_button,
    widgets.HTML("<hr><b>Play Area:</b>"), play_output,
    widgets.HTML("<hr><b>Execution Log:</b>"), full_log_output
]))

print("Play Mode UI and Logic updated (FIX for Combining Interactive Inputs).")